In [1]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np
class ConvAutoencoder:
	@staticmethod
	def build(width, height, depth, filters=(32, 64), latentDim=16):
		# initialize the input shape to be "channels last" along with
		# the channels dimension itself
		# channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1
		# define the input to the encoder
		inputs = Input(shape=inputShape)
		x = inputs
		# loop over the number of filters
		for f in filters:
			# apply a CONV => RELU => BN operation
			x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)
		# flatten the network and then construct our latent vector
		volumeSize = K.int_shape(x)
		x = Flatten()(x)
		latent = Dense(latentDim)(x)
		# build the encoder model
		encoder = Model(inputs, latent, name="encoder")
		# start building the decoder model which will accept the
		# output of the encoder as its inputs
		latentInputs = Input(shape=(latentDim,))
		x = Dense(np.prod(volumeSize[1:]))(latentInputs)
		x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
		# loop over our number of filters again, but this time in
		# reverse order
		for f in filters[::-1]:
			# apply a CONV_TRANSPOSE => RELU => BN operation
			x = Conv2DTranspose(f, (3, 3), strides=2,
				padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)
		# apply a single CONV_TRANSPOSE layer used to recover the
		# original depth of the image
		x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
		outputs = Activation("sigmoid")(x)
		# build the decoder model
		decoder = Model(latentInputs, outputs, name="decoder")
		# our autoencoder is the encoder + decoder
		autoencoder = Model(inputs, decoder(encoder(inputs)),
			name="autoencoder")
		# return a 3-tuple of the encoder, decoder, and autoencoder
		return (encoder, decoder, autoencoder)

2024-01-26 10:39:48.701272: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 10:39:48.701344: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 10:39:48.702517: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-26 10:39:48.791386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist,cifar10
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2

In [13]:
from imutils import paths

In [19]:
import os 
dirs = os.listdir('../IHQ--Output_Segmentation/')

In [25]:
import random
traindirs = random.sample(dirs,int(len(dirs)*0.8))

In [29]:
testdirs = list(set(dirs) - set(traindirs))

In [60]:
trainImages = []
for folder in traindirs:
    for im in list(paths.list_images('../IHQ--Output_Segmentation/'+folder)):
        im1 = cv2.imread(im)
        if((im1.shape)==(64,64,3)):
            trainImages.append(im1)

In [61]:
trainDataset = np.array(trainImages)

In [63]:
f = open('trainDatasetImages', "wb")
f.write(pickle.dumps(trainDataset))
f.close()

In [64]:
testImages = []
testNames = []
for folder in testdirs:
    for im in list(paths.list_images('../IHQ--Output_Segmentation/'+folder)):
        im1 = cv2.imread(im)
        if((im1.shape)==(64,64,3)):
            testNames.append(im)
            testImages.append(im1)
for im in list(paths.list_images('../Malignos--Output_Segmentation/')):
        im1 = cv2.imread(im)
        if((im1.shape)==(64,64,3)):
            testNames.append(im)
            testImages.append(im1)

In [65]:
testDataset = np.array(testImages)

In [67]:
f = open('testDatasetImages', "wb")
f.write(pickle.dumps(testDataset))
f.close()

In [68]:
f = open('testDatasetNames', "wb")
f.write(pickle.dumps(testNames))
f.close()

In [71]:
EPOCHS = 20
INIT_LR = 1e-3
BS = 32
images = trainDataset
images = images.astype("float32") / 255.0
(trainX, validX) = train_test_split(images, test_size=0.2,random_state=42)

In [72]:
def visualize_predictions(decoded, gt, samples=10):
	# initialize our list of output images
	outputs = None
	# loop over our number of output samples
	for i in range(0, samples):
		# grab the original image and reconstructed image
		original = (gt[i] * 255).astype("uint8")
		recon = (decoded[i] * 255).astype("uint8")
		# stack the original and reconstructed image side-by-side
		output = np.hstack([original, recon])
		# if the outputs array is empty, initialize it as the current
		# side-by-side image display
		if outputs is None:
			outputs = output
		# otherwise, vertically stack the outputs
		else:
			outputs = np.vstack([outputs, output])
	# return the output images
	return outputs

In [74]:
print("[INFO] building autoencoder...")
(encoder, decoder, autoencoder) = ConvAutoencoder.build(64, 64, 3)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
autoencoder.compile(loss="mse", optimizer=opt)
# train the convolutional autoencoder
H = autoencoder.fit(
	trainX, trainX,
	validation_data=(validX, validX),
	epochs=EPOCHS,
	batch_size=BS)
# use the convolutional autoencoder to make predictions on the
# testing images, construct the visualization, and then save it
# to disk
print("[INFO] making predictions...")
decoded = autoencoder.predict(validX)
vis = visualize_predictions(decoded, validX)


[INFO] building autoencoder...
Train on 26951 samples, validate on 6738 samples
Epoch 1/20
26951/26951 [==============================] - 23s 866us/sample - loss: 0.0072 - val_loss: 0.0028
Epoch 2/20
26951/26951 [==============================] - 18s 684us/sample - loss: 0.0028 - val_loss: 0.0032
Epoch 3/20
26951/26951 [==============================] - 18s 682us/sample - loss: 0.0026 - val_loss: 0.0021
Epoch 4/20
26951/26951 [==============================] - 18s 679us/sample - loss: 0.0024 - val_loss: 0.0028
Epoch 5/20
26951/26951 [==============================] - 18s 681us/sample - loss: 0.0023 - val_loss: 0.0023
Epoch 6/20
26951/26951 [==============================] - 18s 677us/sample - loss: 0.0022 - val_loss: 0.0023
Epoch 7/20
26951/26951 [==============================] - 18s 681us/sample - loss: 0.0021 - val_loss: 0.0022
Epoch 8/20
26951/26951 [==============================] - 18s 675us/sample - loss: 0.0021 - val_loss: 0.0020
Epoch 9/20
26951/26951 [========================

In [75]:
cv2.imwrite("visualization.jpg", vis)

True

In [76]:
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig('plotCitologia.jpg')
# serialize the autoencoder model to disk
print("[INFO] saving autoencoder...")
autoencoder.save('modelCitologia', save_format="h5")

[INFO] saving autoencoder...
